In [8]:
# Spleen Proximity project
import SimpleITK as sitk
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import numpy as np
import monai
import random
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt

from sklearn.metrics import balanced_accuracy_score
from monai.metrics import ROCAUCMetric
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import RocCurveDisplay, auc
from sklearn.metrics import multilabel_confusion_matrix
from monai.utils import first, set_determinism
from monai.transforms import (
    Activations,
    CenterSpatialCrop,
    EnsureChannelFirst,
    RandRotate90,
    Resize,
    ResizeWithPadOrCrop,
    AsDiscrete,
    Compose,
    LoadImage,
    RandFlip,
    RandRotate,
    RandZoom,
    RandAdjustContrast,
    RandShiftIntensity,
    RandScaleIntensity,
    RandHistogramShift,
    ScaleIntensity,
    ScaleIntensityRange,
    AsDiscrete,
    Activationsd,
    AsDiscreted,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    Crop,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    SaveImaged,
    ScaleIntensity,
    ScaleIntensityRanged,
    ScaleIntensityRangePercentiles,
    SpatialCrop,
    ThresholdIntensity,
    Spacingd,
    Invertd,
    Spacing
)
from monai.handlers.utils import from_engine
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch, ImageDataset, DistributedWeightedRandomSampler
from monai.config import print_config
from monai.apps import download_and_extract
from sklearn.metrics import balanced_accuracy_score

pin_memory = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [9]:
# path to dataset folder
data_dir = '/home/mohammad/projects/ErrolProjects/spleen/dataNifti/abnormal/cropped_segmentor/'
class_names = sorted(x for x in os.listdir(data_dir)
                     if os.path.isdir(os.path.join(data_dir, x)))
class_names = ['AAST13_old', 'AAST45_old', 'negative_old']
num_class = len(class_names)
image_files = [
    [
        os.path.join(data_dir, class_names[i], x)
        for x in random.Random(0).sample(os.listdir(os.path.join(data_dir, class_names[i])), min(608, len(os.listdir(os.path.join(data_dir, class_names[i])))))
    ]
    for i in range(num_class)
]

num_each = [len(image_files[i]) for i in range(num_class)]
image_files_list = []
image_class = []
for i in range(num_class):
    image_files_list.extend(image_files[i])
    image_class.extend([i] * num_each[i])
num_total = len(image_class)


print(f"Total image count: {num_total}")
print(f"Label names: {class_names}")
print(f"Label counts: {num_each}")

Total image count: 1216
Label names: ['AAST13_old', 'AAST45_old', 'negative_old']
Label counts: [455, 153, 608]


In [ ]:
# for roc k-fold
n_splits = 5
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)
fig, ax = plt.subplots(figsize=(6, 6))
input_patch_size = (128, 128, 48)
batch_size = 1
val_frac = 0.2
test_frac = 0
length = len(image_files_list)
indices = np.arange(length)
set_determinism(seed=0)
np.random.shuffle(indices)
spatial_dim = 3
pretrained = False
fold_r = {}
cms = []
n_splits = 5
for fold in range(n_splits):
    val_split = int(val_frac * length)
    val_indices = indices[fold*val_split:(fold+1)*val_split]
    val_x = [image_files_list[i] for i in val_indices]
    val_y = [image_class[i] for i in val_indices]

    val_transforms = Compose(
        [
            EnsureChannelFirst(),
            ScaleIntensityRange(
                a_min=-175,
                a_max=250,
                b_min=0.0,
                b_max=1.0,
                clip=True,
            ),
            Resize(input_patch_size),
        ]
    )

   # create a validation data loader
    val_ds = ImageDataset(image_files=val_x, labels=val_y,
                          transform=val_transforms)
    val_loader = DataLoader(
        val_ds, sampler=None, batch_size=batch_size, num_workers=2, pin_memory=pin_memory)

    y_pred_trans = Compose([Activations(softmax=True)])
    y_trans = Compose([AsDiscrete(to_onehot=num_class)])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = monai.networks.nets.densenet121(spatial_dims=spatial_dim, in_channels=1, pretrained=pretrained,
                                            out_channels=num_class, init_features=128, dropout_prob=0).to(device)

    loss_function = torch.nn.CrossEntropyLoss()

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    val_interval = 1
    auc_metric = ROCAUCMetric()
    auc_metric.average = 'weighted'

    # Test Time Augmentation
    # from monai.data import TestTimeAugmentation

    root_dir = '/home/mohammad/projects/ErrolProjects/spleen/'
    best_metric = -1
    best_acc = -1
    best_metric_epoch = -1
    epoch_loss_values = []
    metric_values = []
    best_model = f"segmentor_normal_low_high_old_121_608samples_f{fold}.pth"

    model.load_state_dict(torch.load(os.path.join(root_dir, best_model)))
    print(best_model)

    model.eval()
    classcount = np.bincount(val_y).tolist()
    val_weights = 1./torch.tensor(classcount, dtype=torch.float)
    val__sampleweights = val_weights[val_y]

    with torch.no_grad():
        y_pred = torch.tensor([], dtype=torch.float32, device=device)
        y = torch.tensor([], dtype=torch.long, device=device)
        for val_data in val_loader:
            val_images, val_labels = (
                val_data[0].to(device),
                val_data[1].to(device),
            )
            y_pred = torch.cat([y_pred, model(val_images)], dim=0)
            y = torch.cat([y, val_labels.long()], dim=0)

        acc_metric_balanced = balanced_accuracy_score(
            np.array(y.cpu()), np.argmax(np.array(y_pred.cpu()), axis=1))

        y_onehot = [y_trans(i) for i in decollate_batch(y)]
        y_pred_act = [y_pred_trans(i) for i in decollate_batch(y_pred)]
        # ROC
        y_score = [i.cpu()[0] for i in y_pred_act]

        viz = RocCurveDisplay.from_predictions(
            np.array(y.cpu()),
            y_score,
            pos_label=0, sample_weight=np.array(val__sampleweights.cpu()),
            name=f"ROC fold {fold}",
            alpha=0.6,
            lw=2,
            ax=ax,
            plot_chance_level=(fold == n_splits - 1),
        )
        interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        aucs.append(viz.roc_auc)

        auc_metric(y_pred_act, y_onehot)
        result = auc_metric.aggregate()
        auc_metric.reset()
        metric_values.append(result)
        acc_value = torch.eq(y_pred.argmax(dim=1), y)
        acc_metric = acc_value.sum().item() / len(acc_value)
        
        print(
            f" current AUC: {result:.4f}"
            f" current accuracy: {acc_metric:.4f}"
            f'\n balanced acc = {acc_metric_balanced}'
        )

        cm = confusion_matrix(np.array(y.cpu()), np.argmax(
            np.array(y_pred.cpu()), axis=1))

        # target_names = ['low-grade','high-grade','normal']
        target_names = ['injured', 'normal']
        cms.append(cm)


        y_test = np.array(y.cpu())
        y_score = np.argmax(np.array(y_pred.cpu()), axis=1)

        fold_r[f'{fold}'] = multilabel_confusion_matrix(
            np.array(y.cpu()), np.argmax(np.array(y_pred.cpu()), axis=1))
        # break

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(
    mean_fpr,
    mean_tpr,
    color="b",
    label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
    lw=2,
    alpha=0.8,
)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(
    mean_fpr,
    tprs_lower,
    tprs_upper,
    color="grey",
    alpha=0.2,
    label=r"$\pm$ 1 std. dev.",
)

ax.set(
    xlim=[-0.05, 1.05],
    ylim=[-0.05, 1.05],
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
    title=f"Mean ROC curve with variability\n(Positive label '{target_names[0]}')",
)
ax.axis("square")
ax.legend(loc="lower right")
plt.show()